In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
%matplotlib inline
ox.config(log_console=True)
weight_by_length = False
plt.style.context('seaborn-bright')
ox.__version__


import fiona
import os,os.path
from shapely.geometry import shape,mapping
from shapely.ops import transform
from functools import partial
import pyproj
from fiona.crs import from_epsg
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import glob
import json

import collections
from pathlib import Path

In [2]:
root= '/datasets_1/sagarj/BellLabs/CitySV_Data/city_centers_sampling/'

In [3]:
places = {
          'London' : 'London, UK',
          'Chicago'       : 'Chicago, IL, USA',
          'Los Angeles'   : {'city':'Los Angeles', 'state':'CA', 'country':'USA'},
          'Manhattan'     : 'Manhattan, NYC, NY, USA',
          'San Francisco' : {'city':'San Francisco', 'state':'CA', 'country':'USA'},
         'Paris':'Paris, France',
         'Tokyo':'Tokyo, Japan',
        ' Berlin':'Berlin, Germany',
         'New York':'New York, NY',
         'Munich':'Munich, Germany',
        'Hamburg':'Hamburg',
         'Moscow':'Moscow',
        'Zurich':'Zurich',
        'Madrid':'Madrid',
        'Vienna':'Vienna',
        'Dusseldorf':'Dusseldorf',
        'Stockholm':'Stockholm',
        'Houston':'Houston',
        'Helsinki':'Helsinki',
    'Frankfurt am Main':'Frankfurt am Main',
    'Toronto':'Toronto'
}

In [4]:
def createPoints(root_Dir, inshp , outshp, mini_dist):
    
    '''
    This function will parse through the street network of provided city and
    clean all highways and create points every mini_dist meters (or as specified) along
    the linestrings
    Required modules: Fiona and Shapely

    parameters:
        inshp: the input linear shapefile, must be in WGS84 projection, ESPG: 4326
        output: the result point feature class
        mini_dist: the minimum distance between two created point

    last modified by Xiaojiang Li, MIT Senseable City Lab
    
    '''
    inshp = root_Dir + inshp
    outshp = root_Dir + outshp
    input_crs = ''
    count = 0
    s = {'trunk_link','tertiary','motorway','motorway_link','steps', None, ' ','pedestrian','primary', 'primary_link','footway','tertiary_link', 'trunk','secondary','secondary_link','tertiary_link','bridleway','service'}
    
    # the temporaray file of the cleaned data
    root = os.path.dirname(root_Dir)
    basename = 'clean_' + os.path.basename(inshp)
    temp_cleanedStreetmap = os.path.join(root,basename)
    c = fiona.open(inshp)
    input_crs = c.crs['init']
    c.close()
    # if the tempfile exist then delete it
    if os.path.exists(temp_cleanedStreetmap):
        fiona.remove(temp_cleanedStreetmap, 'ESRI Shapefile')
    
    # clean the original street maps by removing highways, if it the street map not from Open street data, users'd better to clean the data themselve
    with fiona.open(inshp) as source, fiona.open(temp_cleanedStreetmap, 'w', driver=source.driver, crs=source.crs,schema=source.schema) as dest:
        for feat in source:
            try:
                i = feat['properties']['highway'] # for the OSM street data
                if i in s:
                    continue
            except:
                # if the street map is not osm, do nothing. You'd better to clean the street map, if you don't want to map the GVI for highways
                key = dest.schema['properties'].keys()[0] # get the field of the input shapefile and duplicate the input feature
                i = feat['properties'][key]
                if i in s:
                    continue
            
            dest.write(feat)

    schema = {
        'geometry': 'Point',
        'properties': {'id': 'int'},
    }

    # Create pointS along the streets
    with fiona.drivers():
        #with fiona.open(outshp, 'w', 'ESRI Shapefile', crs=source.crs, schema) as output:
        with fiona.open(outshp, 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = schema) as output:
            for line in fiona.open(temp_cleanedStreetmap):
                first = shape(line['geometry'])
                
                length = first.length
                
                try:
                    # convert degree to meter, in order to split by distance in meter
                    project = partial(pyproj.transform,pyproj.Proj(init=input_crs),pyproj.Proj(init='EPSG:3857')) #3857 is psudo WGS84 the unit is meter
                    
                    line2 = transform(project, first)
                    linestr = list(line2.coords)
                    dist = mini_dist #set
                    for distance in range(0,int(line2.length), dist):
                        point = line2.interpolate(distance)
                        
                        # convert the local projection back the the WGS84 and write to the output shp
                        project2 = partial(pyproj.transform,pyproj.Proj(init='EPSG:3857'),pyproj.Proj(init='EPSG:4326'))
                        point = transform(project2, point)
                        output.write({'geometry':mapping(point),'properties': {'id':1}})
                except:
                    print ("You should make sure the input shapefile is WGS84")
                    return
                    
    print("Process Complete")
    
    # delete the temprary cleaned shapefile
    fiona.remove(temp_cleanedStreetmap, 'ESRI Shapefile')


In [5]:
gdf = ox.geocode_to_gdf(list(places.values()))
gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-0.51038 51.46809, -0.51036 51.46795...",51.691874,51.286760,0.334016,-0.510375,256874343,relation,65606,51.507322,-0.127647,"London, Greater London, England, United Kingdom",boundary,administrative,0.940783
1,"POLYGON ((-87.94010 42.00093, -87.94003 41.998...",42.023040,41.644531,-87.524081,-87.940101,309792632,relation,122604,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States",boundary,administrative,0.961530
2,"POLYGON ((-118.66818 34.18507, -118.66817 34.1...",34.337306,33.659541,-118.155295,-118.668178,311371537,relation,207359,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U...",boundary,administrative,1.073805
3,"MULTIPOLYGON (((-74.04722 40.69086, -74.04722 ...",40.880449,40.683941,-73.906159,-74.047222,312902163,relation,8398124,40.789624,-73.959894,"Manhattan, New York County, New York, United S...",boundary,administrative,0.785439
4,"MULTIPOLYGON (((-123.17382 37.77573, -123.1737...",37.929811,37.640314,-122.281479,-123.173825,256882913,relation,111968,37.779026,-122.419906,"San Francisco, California, United States",boundary,administrative,1.025131
5,"POLYGON ((2.22412 48.85420, 2.22412 48.85402, ...",48.902156,48.815576,2.469760,2.224122,256856867,relation,7444,48.856697,2.351462,"Paris, Ile-de-France, Metropolitan France, France",boundary,administrative,1.051710
6,"MULTIPOLYGON (((135.85369 20.42357, 135.86990 ...",35.898424,20.214581,154.205541,135.853686,257057375,relation,1543125,35.682839,139.759455,"Tokyo, Japan",boundary,administrative,0.979331
7,"MULTIPOLYGON (((13.08835 52.41963, 13.09021 52...",52.675509,52.338245,13.761161,13.088345,256873689,relation,62422,52.501522,13.402550,"Berlin, Germany",boundary,administrative,1.007539
8,"MULTIPOLYGON (((-74.25909 40.49721, -74.25903 ...",40.916179,40.477399,-73.700181,-74.259090,256907472,relation,175905,40.712728,-74.006015,"New York, United States",boundary,administrative,1.017577
9,"MULTIPOLYGON (((11.36078 48.15807, 11.36085 48...",48.248116,48.061624,11.722908,11.360777,256860887,relation,62428,48.137108,11.575382,"Munich, Bavaria, Germany",boundary,administrative,0.966211


In [6]:
gdf = gdf.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [7]:
gdf['centroid'] = gdf['geometry'].centroid

<ipython-input-7-86b40a8bbe24>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


In [8]:
gdf.head(n=2)

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance,centroid
0,"POLYGON ((-0.51038 51.46809, -0.51036 51.46795...",51.691874,51.286760,0.334016,-0.510375,256874343,relation,65606,51.507322,-0.127647,"London, Greater London, England, United Kingdom",boundary,administrative,0.940783,POINT (-0.10941 51.50050)
1,"POLYGON ((-87.94010 42.00093, -87.94003 41.998...",42.023040,41.644531,-87.524081,-87.940101,309792632,relation,122604,41.875562,-87.624421,"Chicago, Cook County, Illinois, United States",boundary,administrative,0.961530,POINT (-87.68569 41.83806)


In [18]:
# %store gdf

Stored 'gdf' (GeoDataFrame)


In [108]:
loc = 20
cityName = gdf.iloc[loc]['display_name']
query = (gdf.iloc[loc]['centroid'].y , gdf.iloc[loc]['centroid'].x)
# query = (37.7574923898256, -122.44307892450824) #For SF the centroid is in the water, so manually entering a co-ordinate
# query = (35.702275058430814, 139.76576974318246) #Same for Tokyo
# query = (60.16979484576442, 24.93814321400679) # same for helsinku
print(cityName , query)
G = ox.graph_from_point(query, network_type='all', dist=1000 )
nodes, edges = ox.graph_to_gdfs(G)

Toronto, Golden Horseshoe, Ontario, Canada (43.722949576957475, -79.38840457344165)


In [103]:
Path(root + cityName +'/shapeFiles').mkdir(parents=True, exist_ok=True)
Path(root + cityName +'/pointsJson').mkdir(parents=True, exist_ok=True)
Path(root + cityName +'/Mapillary_images').mkdir(parents=True, exist_ok=True)

In [104]:
ox.save_graph_shapefile(G,filepath=root + cityName +'/shapeFiles')

In [105]:
sample_root = root+ cityName+'/shapeFiles/'
createPoints(sample_root , 'edges.shp', 'samples.shp',100)

<ipython-input-4-a8c385fbde69>:56: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pypr

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

...

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the

Process Complete


/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/work/sagarj/miniconda3/envs/opencv/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the